# Importing Libraries and Reading Dataset

In [35]:
import numpy as np
from math import sqrt

In [36]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_ab1fee0f25804e008eab74f649a5bf23 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='CBEvzqubzlmtMd2-soX6lwmZ11OQgfzvcpi47i6CuvAR',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_ab1fee0f25804e008eab74f649a5bf23.get_object(Bucket='dataanalysis-donotdelete-pr-pygkr2zsk2cfwa',Key='movies.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

dfm = pd.read_csv(body)
dfm.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [37]:

body = client_ab1fee0f25804e008eab74f649a5bf23.get_object(Bucket='dataanalysis-donotdelete-pr-pygkr2zsk2cfwa',Key='ratings.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

dfr = pd.read_csv(body)
dfr.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


# Cleaning

In [38]:
# seperate genres and year
dfm['year'] = dfm.title.str.extract('(\(\d\d\d\d\))', expand = False)
dfm['year'] = dfm.year.str.extract('(\d\d\d\d)', expand = False)
dfm['title'] = dfm.title.str.replace('(\(\d\d\d\d\))','')
# remove blank spaces
dfm['title'] = dfm['title'].apply(lambda x: x.strip())
dfm.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [39]:
dfm['genres'] = dfm.genres.str.split('|')
dfm.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [40]:
# making dummy variables for genres
dfmg = dfm.copy()
for index,row in dfm.iterrows():
    for g in row['genres']:
        dfmg.at[index,g]=1
dfmg = dfmg.fillna(0)
dfmg.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
#cleaning for dfr
dfr = dfr.drop('timestamp',1)
dfr.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


# Creating Recommendations

In [42]:
# movies not yet watched
recco = [
    {'title':'Showgirls', 'rating':'2.75'},
    {'title':'Akira','rating':'4.1'},
    {'title':'Top Gun','rating':'4'},
    {'title':'Goodfellas','rating':'4.5'},
    {'title':'Pulp Fiction','rating':'4.3'}
]
recomv = pd.DataFrame(recco)
recomv

,rating,title
0,2.75,Showgirls
1,4.1,Akira
2,4,Top Gun
3,4.5,Goodfellas
4,4.3,Pulp Fiction


In [43]:
id_ = dfm[dfm['title'].isin(recomv['title'].tolist())]
# merging by title
recomv =pd.merge(id_,recomv)
recomv = recomv.drop('genres',1).drop('year',1)
recomv

,movieId,title,rating
0,193,Showgirls,2.75
1,296,Pulp Fiction,4.3
2,1101,Top Gun,4
3,1213,Goodfellas,4.5
4,1274,Akira,4.1


In [44]:
# the movies the viewer watched
um = dfmg[dfmg['movieId'].isin(recomv['movieId'].tolist())]
um

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
191,193,Showgirls,[Drama],1995,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
293,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1079,1101,Top Gun,"[Action, Romance]",1986,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1187,1213,Goodfellas,"[Crime, Drama]",1990,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1246,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
# making genre matrix
um = um.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
genmtx = um.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genmtx

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Making recommendation matrix based on weights

In [46]:
recomv['rating'] = recomv['rating'].astype('float')
recomv

,movieId,title,rating
0,193,Showgirls,2.75
1,296,Pulp Fiction,4.30
2,1101,Top Gun,4.00
3,1213,Goodfellas,4.50
4,1274,Akira,4.10


In [47]:
# using dot product to assign weights based on rating
usermtx = genmtx.transpose().dot(recomv['rating'])
usermtx

Adventure              4.10
Animation              4.10
Children               0.00
Comedy                 4.30
Fantasy                0.00
Romance                4.00
Drama                 11.55
Action                 8.10
Crime                  8.80
Thriller               4.30
Horror                 0.00
Mystery                0.00
Sci-Fi                 4.10
IMAX                   0.00
Documentary            0.00
War                    0.00
Musical                0.00
Western                0.00
Film-Noir              0.00
(no genres listed)     0.00
dtype: float64

In [49]:
# genres in original table
genretbl = dfmg.set_index(dfmg['movieId'])
genretbl = genretbl.drop('movieId',1).drop('year',1).drop('title',1).drop('genres',1)
genretbl.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Making Recommendations

In [50]:
# create recommendation score for each user
recommend = (genretbl*usermtx).sum(axis=1)/usermtx.sum()
recommend.head()

movieId
1    0.234302
2    0.076851
3    0.155576
4    0.372071
5    0.080600
dtype: float64

In [51]:
recommend = recommend.sort_values(ascending=False)
recommend.head(10)

movieId
4719      0.846298
75408     0.846298
76153     0.846298
64645     0.771321
122787    0.771321
81132     0.771321
127341    0.769447
26701     0.767573
115479    0.767573
459       0.765698
dtype: float64

In [62]:
recommendation_10 = dfm.loc[dfm['movieId'].isin(recommend.head(20).keys())]
recommendation_10.reset_index(drop=True, inplace =True)
recommendation_10

,movieId,title,genres,year
0,145,Bad Boys,"[Action, Comedy, Crime, Drama, Thriller]",1995
1,459,"Getaway, The","[Action, Adventure, Crime, Drama, Romance, Thr...",1994
2,1432,Metro,"[Action, Comedy, Crime, Drama, Thriller]",1997
3,4719,Osmosis Jones,"[Action, Animation, Comedy, Crime, Drama, Roma...",2001
4,5018,Motorama,"[Adventure, Comedy, Crime, Drama, Fantasy, Mys...",1991
5,5628,Wasabi,"[Action, Comedy, Crime, Drama, Thriller]",2001
6,7007,"Last Boy Scout, The","[Action, Comedy, Crime, Drama, Thriller]",1991
7,7235,Ichi the Killer (Koroshiya 1),"[Action, Comedy, Crime, Drama, Horror, Thriller]",2001
8,26701,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,"[Action, Animation, Crime, Drama, Film-Noir, M...",1989
9,43853,"Business, The","[Action, Comedy, Crime, Drama, Thriller]",2005
